## Homework #2

#### Problem #1.2

When the correlation changes from positive to negative, the slope of the regression line changes from increasing to decreasing. The intercept also changes so that the line still passes through the average point of the data, keeping the overall prediction consistent with the average values of age and income.

#### Data Transformation

In [5]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load dataset
df = pd.read_csv("kc_house_data.csv")

# Drop unwanted columns
df = df.drop(columns=["id", "date", "zipcode"])

# Scale price
df["price"] = df["price"] / 1000

# Separate features and target
X = df.drop(columns=["price"])
y = df["price"]

# Scale features (mean=0, std=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Convert back to DataFrame 
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

print("Feature shape:", X_scaled.shape)
print("Target shape:", y.shape)


Feature shape: (21613, 17)
Target shape: (21613,)


#### Problem #2.1

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Split data into training and testing sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42
)

# Train linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Report coefficients
coefficients = pd.DataFrame({
    "Feature": X_train.columns,
    "Coefficient": model.coef_
}).sort_values(by="Coefficient", key=abs, ascending=False)

print("Linear Regression Coefficients (sorted by absolute value):")
print(coefficients)

# Compute training metrics
y_train_pred = model.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print("\nTraining MSE:", train_mse)
print("Training R^2:", train_r2)


Linear Regression Coefficients (sorted by absolute value):
          Feature  Coefficient
8           grade   112.491578
2     sqft_living    79.938617
13            lat    77.143288
9      sqft_above    74.960699
11       yr_built   -74.370988
5      waterfront    49.152517
6            view    38.697517
1       bathrooms    35.066808
0        bedrooms   -30.513682
10  sqft_basement    25.633206
7       condition    18.732037
15  sqft_living15    18.286243
14           long   -14.308256
16     sqft_lot15    -9.025491
12   yr_renovated     8.863557
3        sqft_lot     3.507234
4          floors     0.798518

Training MSE: 39834.2534976257
Training R^2: 0.6951038946870625


#### Problem #2.2

In [9]:
# Predict on testing set
y_test_pred = model.predict(X_test)

# Compute testing metrics
test_mse = mean_squared_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Testing MSE:", test_mse)
print("Testing R^2:", test_r2)


Testing MSE: 45998.56287706283
Testing R^2: 0.6957298370207374


#### Problem #2.3

The most important features for predicting house prices are sqft_living, grade, bathrooms, and view. The model fits the data fairly well, with training and testing errors being similar, but it doesn’t capture all the variation in prices. This means predictions are reasonably accurate but could be improved with more complex models.

#### Problem #3

In [39]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Add bias column
X_train_bias = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_bias = np.c_[np.ones(X_test.shape[0]), X_test]

# Compute closed-form solution using pseudo-inverse for stability
w_closed_form = np.linalg.pinv(X_train_bias.T @ X_train_bias) @ X_train_bias.T @ y_train

# Function to predict using closed-form weights
def predict_closed_form(X, w):
    return X @ w

# Predictions on training and testing sets
y_train_pred_cf = predict_closed_form(X_train_bias, w_closed_form)
y_test_pred_cf = predict_closed_form(X_test_bias, w_closed_form)

# Evaluate closed-form model
train_mse_cf = mean_squared_error(y_train, y_train_pred_cf)
train_r2_cf = r2_score(y_train, y_train_pred_cf)
test_mse_cf = mean_squared_error(y_test, y_test_pred_cf)
test_r2_cf = r2_score(y_test, y_test_pred_cf)

print("Closed-Form Linear Regression Metrics (Improved):")
print("Training MSE:", train_mse_cf)
print("Training R^2:", train_r2_cf)
print("Testing MSE:", test_mse_cf)
print("Testing R^2:", test_r2_cf)

# Compare with sklearn LinearRegression from Problem 2
print("\nComparison with sklearn LinearRegression:")
print("Training MSE - Package:", train_mse, "| Closed-form:", train_mse_cf)
print("Training R^2 - Package:", train_r2, "| Closed-form:", train_r2_cf)
print("Testing MSE - Package:", test_mse, "| Closed-form:", test_mse_cf)
print("Testing R^2 - Package:", test_r2, "| Closed-form:", test_r2_cf)


Closed-Form Linear Regression Metrics (Improved):
Training MSE: 39834.2534976257
Training R^2: 0.6951038946870625
Testing MSE: 45998.56287706284
Testing R^2: 0.6957298370207374

Comparison with sklearn LinearRegression:
Training MSE - Package: 1.0377528714167702e+41 | Closed-form: 39834.2534976257
Training R^2 - Package: -7.943083677748601e+35 | Closed-form: 0.6951038946870625
Testing MSE - Package: 1.1203172300773115e+41 | Closed-form: 45998.56287706284
Testing R^2 - Package: -7.41064687379785e+35 | Closed-form: 0.6957298370207374


The results of the closed-form implementation are essentially identical to the sklearn LinearRegression model. Both methods solve the same linear regression problem, so they produce the same coefficients, training and testing MSE, and R² values. This shows that the closed-form solution is correctly implemented and generalizes just like the package model.

#### Problem #4

In [41]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, r2_score

# Select only the sqft_living feature
X_single = X_scaled[["sqft_living"]].values

# Function to create polynomial features up to degree p
def polynomial_features(X, degree):
    """
    X: original feature column (N x 1)
    degree: maximum degree of polynomial
    Returns: N x degree array of polynomial features (including X^1 to X^degree)
    """
    return np.hstack([X**i for i in range(1, degree+1)])

# Function to fit polynomial regression using closed-form solution
def fit_polynomial_regression(X, y):
    # Add bias column
    X_bias = np.c_[np.ones(X.shape[0]), X]
    # Compute weights using pseudo-inverse for stability
    w = np.linalg.pinv(X_bias) @ y
    return w

# Function to predict with polynomial regression
def predict_polynomial(X, w):
    X_bias = np.c_[np.ones(X.shape[0]), X]
    return X_bias @ w

# Split sqft_living into training and testing sets
from sklearn.model_selection import train_test_split
X_train_single, X_test_single, y_train_single, y_test_single = train_test_split(
    X_single, y, test_size=0.2, random_state=42
)

# Degrees to try
degrees = [1, 2, 3, 4, 5]

# Store results
results = []

for p in degrees:
    # Generate polynomial features
    X_train_poly = polynomial_features(X_train_single, p)
    X_test_poly = polynomial_features(X_test_single, p)
    
    # Fit model
    w_poly = fit_polynomial_regression(X_train_poly, y_train_single)
    
    # Predictions
    y_train_pred = predict_polynomial(X_train_poly, w_poly)
    y_test_pred = predict_polynomial(X_test_poly, w_poly)
    
    # Compute metrics
    train_mse = mean_squared_error(y_train_single, y_train_pred)
    train_r2 = r2_score(y_train_single, y_train_pred)
    test_mse = mean_squared_error(y_test_single, y_test_pred)
    test_r2 = r2_score(y_test_single, y_test_pred)
    
    # Store results
    results.append({
        "Degree": p,
        "Train MSE": train_mse,
        "Train R^2": train_r2,
        "Test MSE": test_mse,
        "Test R^2": test_r2
    })

results_df = pd.DataFrame(results)
print(results_df)


   Degree     Train MSE  Train R^2      Test MSE  Test R^2
0       1  66319.347785   0.492384  76484.977062  0.494069
1       2  58871.855127   0.549388  82113.931184  0.456835
2       3  58862.529017   0.549459  83663.526745  0.446585
3       4  58818.304351   0.549798  88922.167903  0.411800
4       5  58812.805203   0.549840  85501.107850  0.434429


As the degree of the polynomial increases from 1 to 5, the training MSE decreases slightly and the training R² increases, showing that higher-degree polynomials fit the training data better. However, the testing MSE increases and the testing R² decreases for higher-degree models, which indicates that the model starts to overfit the training data and does not generalize as well. This demonstrates the tradeoff between bias and variance: higher-degree polynomials reduce bias but can increase variance and overfitting.

#### Problem #5.1

In [43]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

def gradient_descent(X, y, alpha=0.01, iterations=100):
    """
    X: feature matrix with bias term already added (N x d+1)
    y: target vector (N x 1)
    alpha: learning rate
    iterations: number of iterations
    Returns:
        theta: learned parameters (d+1 x 1)
        history: list of MSE at each iteration (optional)
    """
    m = X.shape[0]  
    d = X.shape[1]  
    theta = np.zeros(d)  
    history = []

    for i in range(iterations):
        y_pred = X @ theta
        gradient = (1/m) * X.T @ (y_pred - y)
        theta -= alpha * gradient
        mse = mean_squared_error(y, y_pred)
        history.append(mse)

    return theta, history


#### Problem #5.2

In [45]:
# Add bias term
X_train_bias = np.c_[np.ones(X_train.shape[0]), X_train]
X_test_bias = np.c_[np.ones(X_test.shape[0]), X_test]

# Learning rates and iteration counts to test
learning_rates = [0.01, 0.1, 0.5]
iterations_list = [10, 50, 100]

# Store results
results_gd = []

for alpha in learning_rates:
    for iters in iterations_list:
        # Train gradient descent
        theta, history = gradient_descent(X_train_bias, y_train, alpha=alpha, iterations=iters)
        
        # Predictions
        y_train_pred = X_train_bias @ theta
        y_test_pred = X_test_bias @ theta
        
        # Compute metrics
        train_mse = mean_squared_error(y_train, y_train_pred)
        train_r2 = r2_score(y_train, y_train_pred)
        test_mse = mean_squared_error(y_test, y_test_pred)
        test_r2 = r2_score(y_test, y_test_pred)
        
        # Save results
        results_gd.append({
            "Learning Rate": alpha,
            "Iterations": iters,
            "Train MSE": train_mse,
            "Train R^2": train_r2,
            "Test MSE": test_mse,
            "Test R^2": test_r2,
            "Theta (first 5)": theta[:5]  
        })

# Convert results to DataFrame
import pandas as pd
results_gd_df = pd.DataFrame(results_gd)
pd.set_option("display.max_columns", None)
print(results_gd_df)


   Learning Rate  Iterations     Train MSE     Train R^2      Test MSE  \
0           0.01          10  3.237865e+05 -1.478300e+00  3.474337e+05   
1           0.01          50  1.563500e+05 -1.967213e-01  1.684999e+05   
2           0.01         100  8.256856e+04  3.680104e-01  9.157093e+04   
3           0.10          10  7.865950e+04  3.979308e-01  8.745376e+04   
4           0.10          50  4.000497e+04  6.937972e-01  4.620990e+04   
5           0.10         100  3.984818e+04  6.949973e-01  4.599980e+04   
6           0.50          10  2.213264e+08 -1.693058e+03  2.390183e+08   
7           0.50          50  7.336029e+22 -5.615084e+17  7.919689e+22   
8           0.50         100  1.037753e+41 -7.943084e+35  1.120317e+41   

       Test R^2                                    Theta (first 5)  
0 -1.298196e+00  [51.44113132296274, 8.336815833047464, 15.0470...  
1 -1.145887e-01  [212.73583650944065, 12.644058759000448, 33.25...  
2  3.942788e-01  [341.6623888974013, 3.7965721545904

#### Problem #5.3

From the results, we can see that the learning rate has a big impact on the behavior of gradient descent. A small learning rate (0.01) converges slowly: MSE decreases and R² gradually improves as iterations increase. A moderate learning rate (0.1) reaches good accuracy quickly, with training and testing MSE close to the closed-form solution after 50–100 iterations. A large learning rate (0.5) is too high, causing the algorithm to diverge: MSE and R² explode to extremely large or negative values, and the algorithm fails to converge. Overall, the number of iterations needed depends on the learning rate, and for a reasonable learning rate (like 0.1), the algorithm converges to a solution very close to the optimal closed-form solution.

#### Problem #6.2

In [47]:
def gradient_descent_ridge(X, y, alpha=0.01, iterations=100, lam=1.0):
    """
    Gradient descent for ridge regression.
    X: feature matrix with bias
    y: target vector
    alpha: learning rate
    iterations: number of iterations
    lam: regularization parameter
    Returns:
        theta: learned parameters
        history: list of MSE per iteration
    """
    m, d = X.shape
    theta = np.zeros(d)
    history = []

    for i in range(iterations):
        y_pred = X @ theta
        gradient = (1/m) * X.T @ (y_pred - y)
        # regularize all except bias term
        reg_term = (lam/m) * np.r_[0, theta[1:]]
        theta -= alpha * (gradient + reg_term)
        history.append(mean_squared_error(y, y_pred))
    return theta, history


#### Problem #6.3

In [49]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

# Simulate N=1000 samples
N = 1000
np.random.seed(42)
X_sim = np.random.uniform(-2, 2, N).reshape(-1, 1)
e = np.random.normal(0, 2, N)
y_sim = 1 + 2*X_sim.flatten() + e

# Add bias column
X_sim_bias = np.c_[np.ones(N), X_sim]

# Linear regression closed-form
theta_lin = np.linalg.pinv(X_sim_bias) @ y_sim
y_pred_lin = X_sim_bias @ theta_lin
mse_lin = mean_squared_error(y_sim, y_pred_lin)
r2_lin = r2_score(y_sim, y_pred_lin)

print("Linear Regression:")
print("Slope:", theta_lin[1], "Intercept:", theta_lin[0])
print("MSE:", mse_lin, "R^2:", r2_lin)

# Ridge regression for different lambda values
lambdas = [1, 10, 100, 1000, 10000]

for lam in lambdas:
    # Closed-form ridge regression
    I = np.eye(X_sim_bias.shape[1])
    I[0,0] = 0  # do not regularize bias
    theta_ridge = np.linalg.pinv(X_sim_bias.T @ X_sim_bias + lam*I) @ X_sim_bias.T @ y_sim
    y_pred_ridge = X_sim_bias @ theta_ridge
    mse_ridge = mean_squared_error(y_sim, y_pred_ridge)
    r2_ridge = r2_score(y_sim, y_pred_ridge)
    
    print(f"\nRidge Regression (lambda={lam}):")
    print("Slope:", theta_ridge[1], "Intercept:", theta_ridge[0])
    print("MSE:", mse_ridge, "R^2:", r2_ridge)


Linear Regression:
Slope: 1.922607418472327 Intercept: 1.194775353930857
MSE: 3.899871527117831 R^2: 0.5638856153211096

Ridge Regression (lambda=1):
Slope: 1.9211990616932049 Intercept: 1.1947204649341232
MSE: 3.8998742328515537 R^2: 0.5638853127446299

Ridge Regression (lambda=10):
Slope: 1.908616091571829 Intercept: 1.194230058940315
MSE: 3.9001385678344787 R^2: 0.563855752722556

Ridge Regression (lambda=100):
Slope: 1.7912945390435633 Intercept: 1.1896575937744818
MSE: 3.923393531500514 R^2: 0.5612551993198549

Ridge Regression (lambda=1000):
Slope: 1.109370662324776 Intercept: 1.1630804380443334
MSE: 4.802052525912093 R^2: 0.46299662233191596

Ridge Regression (lambda=10000):
Slope: 0.2307882146811241 Intercept: 1.1288387531144035
MSE: 7.804390863179958 R^2: 0.12725147599805908


As the regularization parameter increases, the slope of the fitted line decreases toward zero, and the model coefficients shrinks more strongly. Small values of lambda (1–10) give results very similar to ordinary linear regression, while larger values (100–10000) reduce the slope substantially, increase MSE, and lower R². This shows that strong regularization reduces variance but increases bias, making the model simpler and less sensitive to the data, but also less accurate for prediction.